# Eat Safe, Love

## Notebook Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [6]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}
fields = {"_id":1, "AddressLine1":1, "AddressLine2":1, "AddressLine3":1, "BusinessName":1, "BusinessType":1, "BusinessTypeID":1, 
          "PostCode":1, "RatingDate":1, "RatingValue":1, "geocode":1, "scores":1}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments with hygiene score 20: {count}")

# sort in descending order by Hygiene
sort = [("scores.Hygiene", -1)]

# Display the first document in the results using pprint
print()
pprint(establishments.find(query, fields).sort(sort)[0])


Number of establishments with hygiene score 20: 41

{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingValue': 0,
 '_id': ObjectId('6786f2d15f4e1203dc6f1f4c'),
 'geocode': {'latitude': 50.769705, 'longitude': 0.27694},
 'scores': {'ConfidenceInManagement': 20, 'Hygiene': 20, 'Structural': 20}}


In [7]:
# Convert the result to a Pandas DataFrame
results = establishments.find(query, fields).sort(sort)
df = pd.json_normalize(results)

# Display the number of rows in the DataFrame
print(f"Number of rows in DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
display(df.head(10))


Number of rows in DataFrame: 41


,_id,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,PostCode,RatingValue,RatingDate,scores.Hygiene,scores.Structural,scores.ConfidenceInManagement,geocode.longitude,geocode.latitude
0,6786f2d15f4e1203dc6f1f4c,The Chase Rest Home,Caring Premises,5,5-6 Southfields Road,Eastbourne,East Sussex,BN21 1BU,0,2021-09-23T00:00:00,20,20,20,0.276940,50.769705
1,6786f2d15f4e1203dc6f22cd,Brenalwood,Caring Premises,5,Hall Lane,Walton-on-the-Naze,Essex,CO14 8HN,0,2022-06-22T00:00:00,20,15,30,1.278721,51.857536
2,6786f2d25f4e1203dc6f25d7,Melrose Hotel,Hotel/bed & breakfast/guest house,7842,53 Marine Parade East,Clacton On Sea,Essex,CO15 6AD,0,2021-08-13T00:00:00,20,20,20,1.159270,51.789429
3,6786f2d25f4e1203dc6f27c5,Seaford Pizza,Takeaway/sandwich shop,7844,4 High Street,Seaford,East Sussex,BN25 1PG,1,2021-12-23T00:00:00,20,10,20,0.102020,50.770885
4,6786f2d25f4e1203dc6f27d4,Golden Palace,Restaurant/Cafe/Canteen,1,5 South Street,Seaford,East Sussex,BN25 1HP,1,2021-10-07T00:00:00,20,10,20,0.101446,50.770724
5,6786f2d25f4e1203dc6f3173,Ashby's Butchers,Retailers - other,4613,777 Southchurch Road,Southend-On-Sea,Essex,SS1 2PP,0,2022-07-21T00:00:00,20,20,20,0.736349,51.541448
6,6786f2d25f4e1203dc6f3394,South Sea Express Cuisine,Restaurant/Cafe/Canteen,1,33 Alexandra Street,Southend-on-sea,Essex,SS1 1BW,0,2022-08-03T00:00:00,20,20,20,0.712167,51.535007
7,6786f2d35f4e1203dc6f48bf,Golden Palace,Takeaway/sandwich shop,7844,7 London Road,Rayleigh,Essex,SS6 9HN,0,2022-03-23T00:00:00,20,15,30,0.602364,51.591515
8,6786f2d45f4e1203dc6f4d03,The Tulip Tree,Restaurant/Cafe/Canteen,1,3 The Village,Chiddingstone,KENT,TN8 7AH,1,2022-03-04T00:00:00,20,5,20,0.146450,51.185947
9,6786f2d45f4e1203dc6f5515,F & S,Retailers - other,4613,,81 Southernhay,Basildon,SS14 1EU,0,2021-07-26T00:00:00,20,20,20,0.462307,51.570050


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [8]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {"LocalAuthorityName": {"$regex": "London"}, "RatingValue": {"$gte": 4}}
results = list(establishments.find(query))

# Find the establishments with London in the Local Authority and has a RatingValue greater than or equal to 4.
query = {"LocalAuthorityName": {"$regex": "London"},
        "RatingValue": {"$gte": 4}}
fields = {"_id":1, "AddressLine1":1, "AddressLine2":1, "AddressLine3":1, "BusinessName":1, "BusinessType":1, "BusinessTypeID":1, 
          "PostCode":1, "RatingDate":1, "RatingValue":1, "geocode":1, "scores":1}


# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments in London with RatingValue >= 4: {count}")

# Display the first document in the results using pprint
if results:
    pprint(results[0])


Number of establishments in London with RatingValue >= 4: 33
{'AddressLine1': 'Oak Apple Farm Building 103 Sheernes Docks',
 'AddressLine2': 'Sheppy Kent',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': "Charlie's",
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4627.439467780196,
 'FHRSID': 621707,
 'LocalAuthorityBusinessID': 'PI/000025307',
 'LocalAuthorityCode': '508',
 'LocalAuthorityEmailAddress': 'publicprotection@cityoflondon.gov.uk',
 'LocalAuthorityName': 'City of London Corporation',
 'LocalAuthorityWebSite': 'http://www.cityoflondon.gov.uk/Corporation/homepage.htm',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'ME12',
 'RatingDate': '2021-10-18T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6786f2d25f4e1203dc6f3967'),
 'geocode': {'latitude': 51.369321, 'longitude': 0.508551},
 'links': [{'href': 'https://api.ratings.fo

In [9]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows in DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
display(df.head(10))


Number of rows in DataFrame: 33


,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,LocalAuthorityWebSite,LocalAuthorityEmailAddress,scores,SchemeType,geocode,RightToReply,Distance,NewRatingPending,meta,links
0,6786f2d25f4e1203dc6f3967,621707,0,PI/000025307,Charlie's,Other catering premises,7841,Oak Apple Farm Building 103 Sheernes Docks,Sheppy Kent,,...,http://www.cityoflondon.gov.uk/Corporation/hom...,publicprotection@cityoflondon.gov.uk,"{'Hygiene': 5, 'Structural': 10, 'ConfidenceIn...",FHRS,"{'longitude': 0.508551, 'latitude': 51.369321}",,4627.439468,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings...."
1,6786f2d35f4e1203dc6f3c8d,1130836,0,PI/000034075,Mv City Cruises Erasmus,Other catering premises,7841,Cherry Garden Pier,Cherry Garden Street Rotherhithe,Charlton,...,http://www.cityoflondon.gov.uk/Corporation/hom...,publicprotection@cityoflondon.gov.uk,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': 0.249255999922752, 'latitude': 5...",,4627.873179,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
2,6786f2d35f4e1203dc6f47da,293783,0,PI/000002614,Benfleet Motor Yacht Club,Other catering premises,7841,Ferry Road,South Benfleet Essex,,...,http://www.cityoflondon.gov.uk/Corporation/hom...,publicprotection@cityoflondon.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': 0.561954, 'latitude': 51.543831}",,4631.968418,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
3,6786f2d45f4e1203dc6f55da,1315095,0,PI/000036464,Coombs Catering t/a The Lock and Key,Restaurant/Cafe/Canteen,1,Leslie Ford House,Tilbury,Charlton,...,http://www.cityoflondon.gov.uk/Corporation/hom...,publicprotection@cityoflondon.gov.uk,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': 0.346336990594864, 'latitude': 5...",,4636.846754,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
4,6786f2d45f4e1203dc6f55db,294474,0,PI/000014647,Tilbury Seafarers Centre,Restaurant/Cafe/Canteen,1,Tenants Row Tilbury Docks,Tilbury Essex,,...,http://www.cityoflondon.gov.uk/Corporation/hom...,publicprotection@cityoflondon.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': 0.346325, 'latitude': 51.464078}",,4636.847650,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
5,6786f2d45f4e1203dc6f5ce3,294900,0,PI/000019066,Mv Valulla,Other catering premises,7841,Reeds River Cruises LtdKings ReachRiver Thames...,London,,...,http://www.cityoflondon.gov.uk/Corporation/hom...,publicprotection@cityoflondon.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': 0.287555, 'latitude': 51.504071}",,4640.460834,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
6,6786f2d65f4e1203dc6f7df1,293756,0,PI/000002538,Tereza Joanne,Other catering premises,7841,Funcraft UK Ltd King George V Dock Woolwich Ma...,London,,...,http://www.cityoflondon.gov.uk/Corporation/hom...,publicprotection@cityoflondon.gov.uk,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': 0.069286, 'latitude': 51.501121}",,4648.301822,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
7,6786f2d65f4e1203dc6f81b8,878523,0,PI/000029844,Brick Lane Brews,Restaurant/Cafe/Canteen,1,Air side London City Airport,London,,...,http://www.cityoflondon.gov.uk/Corporation/hom...,publicprotection@cityoflondon.gov.uk,"{'Hygiene': 10, 'Structural': 5, 'ConfidenceIn...",FHRS,"{'longitude': 0.048006, 'latitude': 51.503733}",,4649.173485,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
8,6786f2d6

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [10]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query = {"geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
        "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
        "RatingValue": 5}
fields = {"_id":1, "AddressLine1":1, "AddressLine2":1, "AddressLine3":1, "BusinessName":1, "BusinessType":1, "BusinessTypeID":1, 
          "PostCode":1, "RatingDate":1, "RatingValue":1, "geocode":1, "scores":1}
sort = [("scores.Hygiene", 1)] #, ("BusinessName", 1)]
limit = 5

# Print the results
# Use count_documents to display the number of documents in the result
print(establishments.count_documents(query))

# Display the first document in the results using pprint
print()
pprint(establishments.find(query, fields).sort(sort).limit(limit)[0])

87

{'AddressLine1': '144 - 146 Plumstead High Street',
 'AddressLine2': '',
 'AddressLine3': 'Plumstead',
 'BusinessName': 'Iceland',
 'BusinessType': 'Retailers - supermarkets/hypermarkets',
 'BusinessTypeID': 7840,
 'PostCode': 'SE18 1JQ',
 'RatingDate': '2019-11-13T00:00:00',
 'RatingValue': 5,
 '_id': ObjectId('6786f2d65f4e1203dc6f777e'),
 'geocode': {'latitude': 51.4871482849121, 'longitude': 0.0924199968576431},
 'scores': {'ConfidenceInManagement': 5, 'Hygiene': 0, 'Structural': 5}}


In [11]:
# Convert result to Pandas DataFrame
results = establishments.find(query, fields).sort(sort).limit(limit)
df = pd.json_normalize(results)

# Display the number of rows in the DataFrame
print(df.shape)

# Display the first 10 rows of the DataFrame
df.head(10)


(5, 15)


,_id,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,PostCode,RatingValue,RatingDate,scores.Hygiene,scores.Structural,scores.ConfidenceInManagement,geocode.longitude,geocode.latitude
0,6786f2d65f4e1203dc6f77ab,Volunteer,Pub/bar/nightclub,7843,130 - 132 Plumstead High Street,,Plumstead,SE18 1JQ,5,2019-08-05T00:00:00,0,0,0,0.092080,51.487344
1,6786f2d65f4e1203dc6f77c3,Plumstead Manor Nursery,Caring Premises,5,Plumstead Manor School Old Mill Road,,Plumstead,SE18 1QG,5,2021-06-16T00:00:00,0,0,5,0.085994,51.481518
2,6786f2d65f4e1203dc6f77c6,Atlantic Fish Bar,Takeaway/sandwich shop,7844,35 Lakedale Road,,Plumstead,SE18 1PR,5,2021-06-16T00:00:00,0,0,5,0.091216,51.486730
3,6786f2d65f4e1203dc6f777e,Iceland,Retailers - supermarkets/hypermarkets,7840,144 - 146 Plumstead High Street,,Plumstead,SE18 1JQ,5,2019-11-13T00:00:00,0,5,5,0.092420,51.487148
4,6786f2d65f4e1203dc6f778e,Howe and Co Fish and Chips - Van 17,Mobile caterer,7846,Restaurant And Premises 107A Plumstead High St...,,Plumstead,SE18 1SE,5,2021-11-11T00:00:00,0,0,0,0.092537,51.487534


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [12]:
# Create a pipeline that:
pipeline = [
    {"$match": {"scores.Hygiene": 0}}, # 1. Matches establishments with a hygiene score of 0
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}}, # 2. Groups the matches by Local Authority

    {"$sort": {"count": -1}} # 3. Sorts the matches from highest to lowest

]

# Execute the pipeline
results = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print(f"Number of documents in aggregation result: {len(results)}")

# Print the first 10 results
for res in results[:10]:
    pprint(res)

Number of documents in aggregation result: 55
{'_id': 'Thanet', 'count': 1130}
{'_id': 'Greenwich', 'count': 882}
{'_id': 'Maidstone', 'count': 713}
{'_id': 'Newham', 'count': 711}
{'_id': 'Swale', 'count': 686}
{'_id': 'Chelmsford', 'count': 680}
{'_id': 'Medway', 'count': 672}
{'_id': 'Bexley', 'count': 607}
{'_id': 'Southend-On-Sea', 'count': 586}
{'_id': 'Tendring', 'count': 542}


In [13]:
# Execute the pipeline
results = list(establishments.aggregate(pipeline))


In [14]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows in DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
display(df.head(10))


Number of rows in DataFrame: 55


,_id,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542
